# Beopest example
This notebook sets up a beopest master on a node and uses the hostname for the beopest master to start beopest slaves.
These are then used for submitting a sample .pst and model for an optimization run

In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from dmsbatch import create_batch_client, create_blob_client
import datetime

## First create a batch client from the config file

In [ ]:
client = create_batch_client('../tests/data/dmsbatch.config')
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

In [ ]:
app_pkgs = [('dsm2', '8.2.c5aacef7', 'DSM2-8.2.c5aacef7-win32/bin'),
            ('vista', '1.0-v2019-05-28', 'vista/bin'),
            #('vista', '1.0-v2014-04-28', 'vista/bin'),
            ('unzip', '5.51-1', 'bin'),
           ('pest','17.3','')]

### Show vms available

https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series

In [ ]:
#display(client.skus_available())

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

In [ ]:
pool_name='pestpool'

In [ ]:
client.create_pool(pool_name,
                   27,
                   app_packages=[(app,version) for app,version,_ in app_pkgs], 
                   vm_size='standard_f2s_v2', 
                   tasks_per_vm=1, # keeping only 1 task per vm due to issue with HEC dss vue conflicting with catalogs!
                   os_image_data=('microsoftwindowsserver', 'windowsserver', '2019-datacenter-core')
                  )

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail which is ok as its already there

In [ ]:
job_name='pestjob1'
client.create_job(job_name,pool_name)

## Upload the directory to blob storage
Upload the directory to a blob container with same name as the job

In [ ]:
local_dir = '../tests/data/pest_calib_mtz_ec'
input_file=blob_client.zip_and_upload(job_name,'',local_dir,30)
#
input_file = client.create_input_file_spec(job_name,blob_prefix=input_file.file_path,file_path='.')

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

In [ ]:
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'beopest_master_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path('echo hostname %COMPUTERNAME% & hostname & unzip pest_calib_mtz_ec.zip & call start_pest_master.bat',app_pkgs)
print(task_name)
print(cmd_string)

In [ ]:
pest_task = client.create_task(task_name,cmd_string,resource_files=[input_file])

### Next submit the task for master and wait for hostname to show up in the stdout.txt

In [ ]:
client.submit_tasks(job_name,[pest_task])

In [ ]:
def parse_hostname(client, job_id, task_id):
    stdout=client.read_task_file_as_string(job_id,task_id,'stdout.txt')
    import io
    fh=io.StringIO(stdout)
    line=fh.readline()
    hostname = line.split('hostname')[1].strip()
    return hostname

In [ ]:
import time
while True:
    try:
        hostname = parse_hostname(client, 'pestjob1',pest_task.id)
        break;
    except:
        time.sleep(10)
        continue

In [ ]:
print('Beopest master is running on',hostname)

## Next start the slaves (workers) with the hostname information

In [ ]:
task_name = f'beopest_slave_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path('unzip pest_calib_mtz_ec.zip & call start_pest_slave.bat',app_pkgs)
print(task_name)
print(cmd_string)

In [ ]:
pest_slave_tasks = [client.create_task(task_name+'_%d'%i,
                                      cmd_string,
                                      resource_files=[input_file],
                                      env_settings={'MASTER_HOSTNAME':hostname}) for i in range(1,27)]
client.submit_tasks(job_name,pest_slave_tasks)

## Finally resize the pool to 0 to save costs

In [ ]:
client.resize_pool(pool_name,0)